## Integrin
## DMap and decoder

## setup

In [1]:
import os
import sys
import math
import time
import pickle
import copy
#import mdtraj
import MDAnalysis as mda
#import pyemma
#import pyemma.util.contexts
import networkx as nx
import random
#import parmed as pmd
import acpype
import pandas as pd
from MDAnalysis.analysis import distances


import numpy as np
from scipy.linalg import eigh
from scipy.spatial.distance import pdist
from scipy.interpolate import interp1d
import matplotlib as mpl
from matplotlib import ticker
if os.environ.get('DISPLAY','') == '':
    print('no display found. Using non-interactive Agg backend')
    mpl.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import torch
#import torch.optim as optim
#import torch.nn as nn
#from torchviz import make_dot
#from torch.utils.data import Dataset, TensorDataset, DataLoader
#from torch.utils.data.dataset import random_split
#from tqdm import tqdm

#import deeptime
#from deeptime.decomposition import TICA
#from deeptime.clustering import KMeans
##import mdshare

from sklearn.preprocessing import MinMaxScaler

from jax import numpy as jnp, jit, vmap
from MDAnalysis.analysis import rms #diffusionmap, align, rms

import nglview as nv


no display found. Using non-interactive Agg backend


#### plot settings

In [2]:
showPlots=1
useMagics=1
if useMagics:
    %matplotlib inline
    #%matplotlib notebook
    %load_ext autoreload
    %autoreload 2
    %reload_ext autoreload
    
font = {'weight' : 'normal',
        'size'   : 25}

plt.rc('font', **font)

#### all atom data

In [3]:
universe_aa_trajs = []

file_dir = '/project2/andrewferguson/sivadasetty/doe/analysis-integrin/'

for rep in ['intermediate1_ff/int1_protein_only']: 

    file_aa_traj = file_dir + 'multiple_fake_points_integrin/all-atom/' + rep + '.pdb'

    universe_aa_trajs.append( mda.Universe(file_aa_traj) )
    
    
universe_cg_trajs = []

for i in range(19):
    
    file_cg_traj = file_dir + 'string_mechanisms/deadbolt/initial_cg_frames/fr_beta_int_' + str(i) + '.pdb' 
            
    universe_cg_trajs.append(mda.Universe(file_cg_traj))


/project2/andrewferguson/sivadasetty/doe/elss/lib/python3.9/site-packages/MDAnalysis/topology/PDBParser.py:331: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "
/project2/andrewferguson/sivadasetty/doe/elss/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:451: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"


In [4]:
universe_aa_trajs[0].select_atoms('name CA').indices

array([    4,    23,    37, ..., 26863, 26887, 26894])

#### Generate target PDB files -- based on both bent mapping

In [50]:
### create a universe for CG model [from example in MDAnalysis]


bent_select_map = 'ba_bent'
universe_index = 0

file_dir = '/project2/andrewferguson/sivadasetty/doe/analysis-integrin/'

bent_aa_cg_map = pd.read_csv(file_dir + 'multiple_fake_points_integrin/cg_mapping/' + bent_select_map + '.dat', names=['map'])

#print(bent_aa_cg_map)


bent_indices = []
for i in range(1, 301):
    
    ## get atoms for each CG site
    bent_idx = np.where(bent_aa_cg_map.map == i)
    ## get one of the atom index (central) in each CG site
    bent_indices.append(bent_idx[0][len(bent_idx[0])//2])
    #cg_pos_frame_0.append( universe_aa_trajs[universe_index].trajectory[0].positions[idx].mean(axis=0) )
    
    
n_residues = 300
n_atoms = n_residues
# create resindex list
resindices = np.arange(n_residues)
# all water molecules belong to 1 segment
segindices = [0] * n_residues
  

### topology attributes
from MDAnalysis.coordinates.memory import MemoryReader

target_cg_universe_list = []
for i in range(19):

    # create the Universe
    target_cg_universe = mda.Universe.empty(n_atoms,n_residues=n_residues,
                                            atom_resindex=resindices,residue_segindex=segindices,
                                            trajectory=True) # necessary for adding coordinates
    
    target_cg_universe.add_TopologyAttr('name', ['CG']*n_residues)
    target_cg_universe.add_TopologyAttr('type', ['CG']*n_residues)
    
    mapping_on = 'bent' 
    ## TWO MAPPINGS: This is for determing CA ATOM indices to match in RMSD calculation.
    ## BENT and INTERMEDIATE have the same mapping.
    if mapping_on == 'bent':
        aa_traj_index = 0
        indices = bent_indices    
    else:
        raise Exception("INVALID OPTION")    

    target_cg_universe.add_TopologyAttr('resid', indices)
    target_cg_universe.add_TopologyAttr('id', universe_aa_trajs[aa_traj_index].select_atoms('name CA').indices[indices]+1)
    
    # For finding residues (ignore previous one)
    #target_cg_universe.add_TopologyAttr('resid', universe_aa_trajs[aa_traj_index].select_atoms('name CA').resids[indices])
    
    cg_pos_frame_0 = universe_cg_trajs[i].select_atoms('name CG').positions
        
    target_cg_universe.atoms.positions = cg_pos_frame_0
                
    target_cg_universe_list.append(target_cg_universe)
    

    

In [51]:
#target_cg_universe_list[0].atoms.resids
#universe_aa_trajs[0].atoms.resids #select_atoms('name CA').resids[indices]

#### Check indices of aa atoms corresponding to cg sites used for distance calculations

In [56]:

print('ag1')
print('index ' + ' '.join([ str(target_cg_universe_list[0].atoms.ids[x]) for x in np.arange(155, 169) ]))
print('\n ag2')
print('index ' + ' '.join([ str(target_cg_universe_list[0].atoms.ids[x]) for x in np.arange(285, 300) ]))
print('\n ag3')
print('index ' + ' '.join([ str(target_cg_universe_list[0].atoms.ids[x]) for x in np.arange(0, 65) ]))
print('\n ag4')
print('index ' + ' '.join([ str(target_cg_universe_list[0].atoms.ids[x]) for x in np.arange(259, 282) ]))


ag1
index 14621 14756 14821 14918 15001 15087 15173 15213 15239 15273 15288 15312 15327 15342

 ag2
index 26204 26290 26357 26429 26502 26570 26608 26655 26713 26737 26762 26796 26829 26864 26895

 ag3
index 57 149 253 379 465 550 621 686 761 865 928 1019 1135 1224 1381 1513 1645 1789 1896 2021 2139 2241 2365 2519 2657 2739 2812 2907 2984 3053 3116 3218 3325 3425 3512 3596 3711 3859 3958 4044 4145 4248 4338 4414 4481 4542 4621 4706 4804 4881 4929 5029 5141 5231 5361 5540 5742 5871 5983 6119 6230 6328 6428 6527 6600

 ag4
index 24438 24573 24684 24764 24817 24873 24924 24985 25075 25140 25216 25279 25334 25416 25475 25515 25545 25601 25649 25698 25753 25823 25888


#### discard: initial target MD set up (mapping indices of RMSD calculations)

In [6]:
#'index ' + ' '.join([ str(target_cg_universe.atoms.ids[x]-1) for x in np.arange(0, 300) ])
#'id ' + ' '.join([ str(target_cg_universe_list[0].atoms.ids[x]) for x in np.arange(169, 300) ])

print('index ' + ' '.join([ str(target_cg_universe_list[0].atoms.ids[x]) for x in np.arange(0, 65) ]))
print('index ' + ' '.join([ str(target_cg_universe_list[0].atoms.ids[x]) for x in np.arange(65, 120) ]))
print('index ' + ' '.join([ str(target_cg_universe_list[0].atoms.ids[x]) for x in np.arange(120, 155) ]))
print('index ' + ' '.join([ str(target_cg_universe_list[0].atoms.ids[x]) for x in np.arange(155, 169) ]))


print('index ' + ' '.join([ str(target_cg_universe_list[0].atoms.ids[x]) for x in np.arange(169, 230) ]))
print('index ' + ' '.join([ str(target_cg_universe_list[0].atoms.ids[x]) for x in np.arange(230, 251) ]))
print('index ' + ' '.join([ str(target_cg_universe_list[0].atoms.ids[x]) for x in np.arange(251, 280) ]))
print('index ' + ' '.join([ str(target_cg_universe_list[0].atoms.ids[x]) for x in np.arange(280, 300) ]))



index 57 149 253 379 465 550 621 686 761 865 928 1019 1135 1224 1381 1513 1645 1789 1896 2021 2139 2241 2365 2519 2657 2739 2812 2907 2984 3053 3116 3218 3325 3425 3512 3596 3711 3859 3958 4044 4145 4248 4338 4414 4481 4542 4621 4706 4804 4881 4929 5029 5141 5231 5361 5540 5742 5871 5983 6119 6230 6328 6428 6527 6600
index 6664 6735 6808 6884 6978 7084 7175 7263 7350 7394 7467 7537 7581 7625 7673 7750 7846 7919 7954 7988 8049 8113 8207 8318 8447 8567 8673 8760 8869 8965 9022 9080 9126 9183 9255 9327 9413 9499 9584 9642 9732 9832 9931 10013 10135 10254 10381 10494 10583 10666 10732 10841 10948 11018 11086
index 11166 11231 11332 11408 11490 11581 11689 11815 11915 11992 12108 12197 12344 12487 12614 12678 12723 12773 12833 12913 13009 13079 13208 13342 13451 13577 13693 13810 13924 14036 14137 14241 14330 14443 14536
index 14621 14756 14821 14918 15001 15087 15173 15213 15239 15273 15288 15312 15327 15342
index 15388 15491 15585 15672 15751 15828 15895 15977 16063 16124 16208 16284 1635

In [7]:
#### save to memory
file_dir = '/project2/andrewferguson/sivadasetty/doe/analysis-integrin/'

for i in range(19):

    cg_atoms = target_cg_universe_list[i].select_atoms("name CG")
    cg_atoms.write(file_dir+'string_mechanisms/deadbolt/initial_cg_frames/fr_beta_int_resmapped_'+str(i)+'.pdb',reindex=False)
    

/project2/andrewferguson/sivadasetty/doe/elss/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:775: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/project2/andrewferguson/sivadasetty/doe/elss/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:1151: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/project2/andrewferguson/sivadasetty/doe/elss/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:1151: UserWarning: Found no information for attr: 'resnames' Using default value of 'UNK'
  warnings.warn("Found no information for attr: '{}'"
/project2/andrewferguson/sivadasetty/doe/elss/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:1151: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/project2/andrewfergu

In [8]:
'id ' + ' '.join([ str(target_cg_universe_list[0].atoms.ids[i]) for i in np.arange(0, 169) ])

'id 57 149 253 379 465 550 621 686 761 865 928 1019 1135 1224 1381 1513 1645 1789 1896 2021 2139 2241 2365 2519 2657 2739 2812 2907 2984 3053 3116 3218 3325 3425 3512 3596 3711 3859 3958 4044 4145 4248 4338 4414 4481 4542 4621 4706 4804 4881 4929 5029 5141 5231 5361 5540 5742 5871 5983 6119 6230 6328 6428 6527 6600 6664 6735 6808 6884 6978 7084 7175 7263 7350 7394 7467 7537 7581 7625 7673 7750 7846 7919 7954 7988 8049 8113 8207 8318 8447 8567 8673 8760 8869 8965 9022 9080 9126 9183 9255 9327 9413 9499 9584 9642 9732 9832 9931 10013 10135 10254 10381 10494 10583 10666 10732 10841 10948 11018 11086 11166 11231 11332 11408 11490 11581 11689 11815 11915 11992 12108 12197 12344 12487 12614 12678 12723 12773 12833 12913 13009 13079 13208 13342 13451 13577 13693 13810 13924 14036 14137 14241 14330 14443 14536 14621 14756 14821 14918 15001 15087 15173 15213 15239 15273 15288 15312 15327 15342'

In [15]:
#### ON SHELL --- FIRST SET EACH TARGET PDB TO APPROPRIATE (based on INT/BENT/OPEN) BOX.
#bent_indices
aa_traj_index

0

In [9]:
#### save to memory

for i in range(19):
    box_u = mda.Universe(file_dir+'string_mechanisms/deadbolt/initial_cg_frames/mdanalysis_files/fr_beta_int_resmapped_'+str(i)+'_box.pdb')

    box_u.add_TopologyAttr('tempfactors')
    box_u.atoms.tempfactors = np.ones(np.shape(box_u.atoms.tempfactors))
    #print(box_u.atoms.tempfactors)
    
    mapping_on = 'bent'
    if mapping_on == 'bent':
        aa_traj_index = 0
        indices = bent_indices
    else:
        raise Exception("INVALID OPTION")
                
    box_u.add_TopologyAttr('id', universe_aa_trajs[aa_traj_index].select_atoms('name CA').indices[indices]+1)
        
    pro = box_u.select_atoms('id ' + ' '.join([ str(target_cg_universe_list[i].atoms.ids[j]) for j in np.arange(0, 300) ]))
    
    proa = box_u.select_atoms('id ' + ' '.join([ str(target_cg_universe_list[i].atoms.ids[j]) for j in np.arange(0, 169) ]))
    
    proa0 = box_u.select_atoms('id ' + ' '.join([ str(target_cg_universe_list[i].atoms.ids[j]) for j in np.arange(0, 65) ]))
    #proa1 = box_u.select_atoms('id ' + ' '.join([ str(target_cg_universe_list[i].atoms.ids[j]) for j in np.arange(65, 120) ]))
    #proa2 = box_u.select_atoms('id ' + ' '.join([ str(target_cg_universe_list[i].atoms.ids[j]) for j in np.arange(120, 155) ]))
    #proa3 = box_u.select_atoms('id ' + ' '.join([ str(target_cg_universe_list[i].atoms.ids[j]) for j in np.arange(155, 169) ]))
    
    prob = box_u.select_atoms('id ' + ' '.join([ str(target_cg_universe_list[i].atoms.ids[j]) for j in np.arange(169, 300) ]))
    
    prob0 = box_u.select_atoms('id ' + ' '.join([ str(target_cg_universe_list[i].atoms.ids[j]) for j in np.arange(169, 230) ]))
    #prob1 = box_u.select_atoms('id ' + ' '.join([ str(target_cg_universe_list[i].atoms.ids[j]) for j in np.arange(230, 251) ]))
    #prob2 = box_u.select_atoms('id ' + ' '.join([ str(target_cg_universe_list[i].atoms.ids[j]) for j in np.arange(251, 280) ]))
    #prob3 = box_u.select_atoms('id ' + ' '.join([ str(target_cg_universe_list[i].atoms.ids[j]) for j in np.arange(280, 300) ]))
    
    
    ## RENAME previous custom -- execute only when needed.
#     if os.path.isfile(file_dir+'string_mechanisms/deadbolt/initial_cg_frames/fr_beta_int_resmapped_'+str(i)+'_box_proa.pdb'):
#         os.rename('multiple_fake_points_integrin/'+state_list[x]+'/target_cg_resmapped_'+str(rep)+'_box_proa.pdb',
#                   'multiple_fake_points_integrin/'+state_list[x]+'/target_cg_resmapped_'+str(rep)+'_box_init_proa.pdb')
#     if os.path.isfile('multiple_fake_points_integrin/'+state_list[x]+'/target_cg_resmapped_'+str(rep)+'_box_prob.pdb'):
#         os.rename('multiple_fake_points_integrin/'+state_list[x]+'/target_cg_resmapped_'+str(rep)+'_box_prob.pdb',
#                   'multiple_fake_points_integrin/'+state_list[x]+'/target_cg_resmapped_'+str(rep)+'_box_init_prob.pdb')
    
    pro.write(file_dir+'string_mechanisms/deadbolt/initial_cg_frames/mdanalysis_files/fr_beta_int_resmapped_'+str(i)+'_box_renum.pdb',reindex=False)
    
    proa.write(file_dir+'string_mechanisms/deadbolt/initial_cg_frames/mdanalysis_files/fr_beta_int_resmapped_'+str(i)+'_box_proa.pdb',reindex=False)
    
    proa0.write(file_dir+'string_mechanisms/deadbolt/initial_cg_frames/mdanalysis_files/fr_beta_int_resmapped_'+str(i)+'_box_proa0.pdb',reindex=False)
    #proa1.write(file_dir+'string_mechanisms/deadbolt/initial_cg_frames/mdanalysis_files/fr_beta_int_resmapped_'+str(i)+'_box_proa1.pdb',reindex=False)
    #proa2.write(file_dir+'string_mechanisms/deadbolt/initial_cg_frames/mdanalysis_files/fr_beta_int_resmapped_'+str(i)+'_box_proa2.pdb',reindex=False)
    #proa3.write(file_dir+'string_mechanisms/deadbolt/initial_cg_frames/mdanalysis_files/fr_beta_int_resmapped_'+str(i)+'_box_proa3.pdb',reindex=False)
    
    prob.write(file_dir+'string_mechanisms/deadbolt/initial_cg_frames/mdanalysis_files/fr_beta_int_resmapped_'+str(i)+'_box_prob.pdb',reindex=False)

    prob0.write(file_dir+'string_mechanisms/deadbolt/initial_cg_frames/mdanalysis_files/fr_beta_int_resmapped_'+str(i)+'_box_prob0.pdb',reindex=False)
    #prob1.write(file_dir+'string_mechanisms/deadbolt/initial_cg_frames/mdanalysis_files/fr_beta_int_resmapped_'+str(i)+'_box_prob1.pdb',reindex=False)
    #prob2.write(file_dir+'string_mechanisms/deadbolt/initial_cg_frames/mdanalysis_files/fr_beta_int_resmapped_'+str(i)+'_box_prob2.pdb',reindex=False)
    #prob3.write(file_dir+'string_mechanisms/deadbolt/initial_cg_frames/mdanalysis_files/fr_beta_int_resmapped_'+str(i)+'_box_prob3.pdb',reindex=False)
    

In [18]:
#proa.atoms.